In [6]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%load_ext autoreload
%autoreload 2
%matplotlib inline
from matplotlib import pyplot as plt
from feat_desc import *
import cv2
import numpy as np
np.set_printoptions(edgeitems=30, linewidth=100000,formatter=dict(float=lambda x: "%.3g" % x))
from corner_detector import *
from anms import *
from feat_match import *
from ransac_est_homography import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ModuleNotFoundError: No module named 'cv2'

In [2]:
img1=cv2.imread('small-right.jpg')
# img1=cv2.imread('right.jpg')

# img1=cv2.imread('rot-middle.jpg')
gray = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
c = corner_detector(gray)
x,y,rmax=anms(c, 3000)

d1 = feat_desc(gray,x,y)
kp1=[]
for (_x,_y) in zip(x,y):
    kp1.append(cv2.KeyPoint(_x,_y,40))


132258


In [ ]:
img2=cv2.imread('small-middle.jpg')
# img2=cv2.imread('middle.jpg')

gray = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
c = corner_detector(gray)
x,y,rmax=anms(c, 3000)
d2 = feat_desc(gray,x,y)
kp2=[]
for (_x,_y) in zip(x,y):
    kp2.append(cv2.KeyPoint(_x,_y,40))


177412


In [ ]:
m,dMatch=feat_match(d1, d2)
x1=[]
y1=[]
x2=[]
y2=[]
for k,idx in enumerate(m):
    if (idx != -1):
        # is 0 x or 1 x?
        x1.append(kp1[idx].pt[0])
        y1.append(kp1[idx].pt[1])
        x2.append(kp2[k].pt[0])
        y2.append(kp2[k].pt[1])
# print(x1)
# print(x2)
# print(len(dMatch))
print(len(x1))
x1=np.array(x1)
x2=np.array(x2)
y1=np.array(y1)
y2=np.array(y2)

In [ ]:
H, inlier_ind=ransac_est_homography(x1,y1,x2,y2,0.5)

In [ ]:
print(H)
mask = np.array(inlier_ind, dtype=bool)
print(mask)
print(x1*mask)
print(y1*mask)
stack = np.vstack((x1*mask,y1*mask,np.ones((x1.shape))))
res = H@stack
print(res[0]/res[2])
print(res[1]/res[2])

stack = np.vstack((x2*mask,y2*mask,np.ones((x1.shape))))
print(stack)

In [ ]:
result = cv2.warpPerspective(img1, H,(img1.shape[1], img1.shape[0]))#,flags=cv2.INTER_LINEAR)
fig=plt.figure(figsize=(16, 12), dpi= 80, facecolor='w', edgecolor='k')

plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
mfilter = []
for idx,i in enumerate(mask):
    if i == True:
        mfilter.append(dMatch[idx])

In [ ]:
img_matches = np.empty((max(img1.shape[0], img2.shape[0]), img1.shape[1]+img2.shape[1], 3), dtype=np.uint8)
cv2.drawMatches(img1, kp1, img2, kp2, mfilter, img_matches, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
# cv2.drawMatches(img1, kp1, img2, kp2, dMatch, img_matches, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

In [ ]:
fig=plt.figure(figsize=(20, 18), dpi= 80, facecolor='w', edgecolor='k')
plt.imshow(cv2.cvtColor(img_matches, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
fig=plt.figure(figsize=(16, 12), dpi= 80, facecolor='w', edgecolor='k')
plt.imshow(img)
plt.scatter(x, y)
plt.axis('off')

In [ ]:
d = feat_desc(gray,x,y)

In [ ]:
for i in d:
    print(i)

In [ ]:
x = [[4,5,6],[1,2,3]]
res=np.linalg.norm(x,axis=1)
res.argsort()
print(res.argsort())